## Deploy your pre-trained keras model to AWS
adapted from https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/

In [ ]:
import numpy as np

## 0. Upload your model to SageMaker
This is a manual step. Feel free to use my model below but otherwise, upload your models from the __jupyter home screen__ using the `upload` button

In [ ]:
#! wget https://raw.githubusercontent.com/liampearson/Youtube/master/Keras%20to%20aws%20SageMaker/models/model.h5

## 1. input the model names
complete either:
- MODEL_LOCATION.  or
- both of JSON and WEIGHTS_LOCATION

In [ ]:
# if your model is saved as only a .h5 file
MODEL_LOCATION =''

# or if your model is saved as 2 files: model as a .json file, and weights as a .h5 file
JSON_LOCATION = ''
WEIGHTS_LOCATION = ''

## 2. Load Your Model
Simply run the cell below; the model will be loaded based on how you defined the above

In [ ]:
if MODEL_LOCATION!='': #if your model is saved as a .h5 file only
    from keras.models import load_model
    model = load_model(MODEL_LOCATION) #load the model
    print("loaded model from MODEL_LOCATION")
    
elif JSON_LOCATION!='': # you have your model saved as a JSON file AND weights
#adapted from https://machinelearningmastery.com/save-load-keras-deep-learning-models/
    from keras.models import model_from_json
    json_file = open(JSON_LOCATION, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(WEIGHTS_LOCATION)
    print("loaded model from JSON_LOCATION and WEIGHTS_LOCATION")

## 3. Convert the Keras Model to the format AWS wants
- Converts to a Protobuff file
- Saves it in a certain aws file structure
- Tarballs this file and zips it

In [ ]:
def convert_h5_to_aws(loaded_model):
    """
    given a pre-trained keras model, this function converts it to a TF protobuf format
    and saves it in the file structure which aws expects
    """  
    from tensorflow.python.saved_model import builder
    from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
    from tensorflow.python.saved_model import tag_constants
    
    # This is the file structure which AWS expects. Cannot be changed. 
    model_version = '1'
    export_dir = 'export/Servo/' + model_version
    
    # Build the Protocol Buffer SavedModel at 'export_dir'
    builder = builder.SavedModelBuilder(export_dir)
    
    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})
    
    from keras import backend as K
    with K.get_session() as sess:
        # Save the meta graph and variables
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
        builder.save()
    
    #create a tarball/tar file and zip it
    import tarfile
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)
        
convert_h5_to_aws(model)

## 4. Move the tarball (tar.gz) to S3

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

This is the name of the bucket which SageMaker made in S3

In [ ]:
# where did it upload to?
print("Bucket name is:")
sagemaker_session.default_bucket()

## 5. Create an instance of a Amazon SageMaker TF Model
First, create an empty train.py file (TensorFlowModel expects this at its 'entry point', but can be empty)

In [ ]:
!touch train.py #create an empty python file

In [ ]:
%%timeit
import boto3, re
from sagemaker import get_execution_role

# the (default) IAM role you created when creating this notebook
role = get_execution_role()

# Create a Sagemaker model (see AWS console>SageMaker>Models)
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

## 6a) Host the model in SageMaker and 
## 6b) Create an Endpoint to access the model 

Deploy the model. This can take ~10 minutes

Ignore the message `update_endpoint is a no-op in sagemaker>=2`

In [ ]:
%%timeit
# Deploy a SageMaker to an endpoint
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

In [ ]:
# What is our endpoint called?
predictor.endpoint

## Success! You have deployed a keras model into AWS
# ---------------------
### 7. Confirm its working correctly by making a prediction
Now, we want to use our endpoint/model. Create a predictor which uses the endpoint

This step depends on what inputs your model is expecting. I simply used the iris dataset and so can feed it 4 inputs of which it will give me 3 probabilities - 1 for each iris type. 

Before deploying to aws, I got the predictions of my model - __locally__ - so that I could compare the local vs aws results (they should be the same). 

Locally, with the input below we get the following predictions:
expected predictions:

- 0.99930930
- 0.00069067377
- 0.00000000000000015728773

In [ ]:
# Create a predictor which uses this new endpoint
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel

predictor=sagemaker.tensorflow.model.TensorFlowPredictor(predictor.endpoint, sagemaker_session)

# .predict send the data to our endpoint
#data = np.asarray([[5. , 3.5, 1.3, 0.3]])
#predictor.predict(data)

## Cleanup!

else you will incur extra charges

https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html

- Stop Notebook
- delete endpoints
- delete models
- delete S3 bucket
- delete cloudwatch groups?